In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import platform
import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.core.nanops import bottleneck_switch

from Detector_Classes.M3RefTracking import M3RefTracking
from Detector_Classes.DetectorConfigLoader import DetectorConfigLoader
from Detector_Classes.DreamDetector import DreamDetector

from det_classes_test import plot_ray_hits_2d, align_dream, get_residuals, get_efficiency, get_efficiency_1d, get_residuals_subdets_with_err, get_rays_in_sub_det_vectorized, get_rays_in_sub_det, get_raw_residuals, get_circle_scan, plot_2D_circle_scan

In [3]:
# Check if platform is Windows or Linux
if platform.system() == 'Windows':
    base_dir = 'F:/Saclay/cosmic_data/'
    det_type_info_dir = 'C:/Users/Dylan/PycharmProjects/Cosmic_Bench_DAQ_Control/config/detectors/'
    out_dir = 'F:/Saclay/Analysis/Cosmic Bench/11-5-24/'
    chunk_size = 5  # Number of files to process at once
elif platform.system() == 'Linux' and platform.node() == 'dylan-MS-7C84':
    base_dir = '/media/ucla/Saclay/cosmic_data/'
    det_type_info_dir = '/home/dylan/PycharmProjects/Cosmic_Bench_DAQ_Control/config/detectors/'
    out_dir = '/media/ucla/Saclay/Analysis/Cosmic Bench/11-5-24/'
    chunk_size = 5  # Number of files to process at once
elif platform.system() == 'Linux' and platform.node() == 'dphnpct50.extra.cea.fr':
    base_dir = '/local/home/dn277127/Bureau/beam_test_25/'
    det_type_info_dir = '/local/home/dn277127/PycharmProjects/Cosmic_Bench_DAQ_Control/config/detectors/'
    out_dir = '/local/home/dn277127/Bureau/cosmic_data/Analysis/11-15-24/'
    chunk_size = 0.2  # Number of files to process at once
else:
    raise OSError('Unknown platform')

run_name = 'run_146'
run_dir = f'{base_dir}{run_name}/'
sub_run_name = 'rotation_-60_test_0'

det_single = 'rd5_strip_esl_1'
# det_single = 'rd5_grid_esl_1'

# file_nums = list(range(0, 3))
# file_nums = list(range(0, 109))
file_nums = [0]
# file_nums = 'all'

noise_sigma = 4

# realign_dream = True  # If False, read alignment from file, if True, realign Dream detector
realign_dream = False  # If False, read alignment from file, if True, realign Dream detector

run_json_path = f'{run_dir}run_config.json'
data_dir = f'{run_dir}{sub_run_name}/filtered_root/'
ped_dir = f'{run_dir}{sub_run_name}/decoded_root/'
m3_dir = f'{run_dir}{sub_run_name}/m3_tracking_root/'
alignment_dir = f'{run_dir}alignments/'

z_align_range = [15, 15]  # mm range to search for optimal z position
z_rot_align_range = [-2, 2]

try:
    os.mkdir(alignment_dir)
except FileExistsError:
    pass
try:
    os.mkdir(out_dir)
except FileExistsError:
    pass

In [4]:
det_config_loader = DetectorConfigLoader(run_json_path, det_type_info_dir)

In [5]:
det_config = det_config_loader.get_det_config(det_single, sub_run_name=sub_run_name)
det = DreamDetector(config=det_config)
print(f'FEU Num: {det.feu_num}')
print(f'FEU Channels: {det.feu_connectors}')
print(f'HV: {det.hv}')

det.load_dream_data(data_dir, ped_dir, noise_sigma, file_nums, chunk_size, hist_raw_amps=True, save_waveforms=False, waveform_fit_func='parabola_vectorized')
print(f'Hits shape: {det.dream_data.hits.shape}')

Found 2 groups in detector map
FEU Num: 2
FEU Channels: [5, 6, 7, 8]
HV: {'drift': 400, 'resist_1': 580, 'resist_2': 540}
Error: No ped files found.
Reading in data...
Getting sub chunk 1/5, events: 0-3896 of 19482


100%|██████████| 1/1 [00:00<00:00,  1.63it/s]

Data Raw Shape: (3896, 512, 24)


Read in data shape: (3896, 256, 24)
Getting amplitudes...


100%|██████████| 7/7 [00:00<00:00, 189.74it/s]


...

100%|██████████| 7/7 [00:00<00:00, 193.26it/s]


Fitting time: 0.08492374420166016 s
Processed sub-chunk 4/5
Getting sub chunk 5/5, events: 15585-19482 of 19482


  0%|          | 0/1 [00:00<?, ?it/s]

Data Raw Shape: (3897, 512, 24)


KeyboardInterrupt: 

In [ ]:
det.dream_data.plot_noise_metric()

In [ ]:
det.dream_data.filter_sparks(spark_filter_sigma=8)

In [ ]:
det.dream_data.plot_noise_metric()

In [ ]:
det.dream_data.plot_hits_vs_strip(print_dead_strips=True)

In [ ]:
det.dream_data.plot_raw_amps_2d_hist()

In [ ]:
det.dream_data.plot_raw_amps_2d_hist(combine_y=2)

In [ ]:
det.dream_data.plot_raw_amps_2d_hist(combine_y=3)

In [ ]:
channel_range = (200, 250)
amplitude_range = (2000, 2800)
det.dream_data.plot_raw_amps_2d_hist(combine_y=10)

# Add a rectangle around channel/amplitude range
rect = plt.Rectangle((channel_range[0], amplitude_range[0]), channel_range[-1] - channel_range[0], amplitude_range[-1] - amplitude_range[0], fill=False, color='red', linewidth=2)
plt.gca().add_patch(rect)

In [ ]:
det.make_sub_detectors()

In [ ]:
det.dream_data.plot_amplitudes_vs_strip()

In [ ]:
# det.make_sub_detectors()
event_nums = det.plot_xy_amp_sum_vs_event_num(True, 500, False, 15)

# if realign_dream:
#     det.center[2] = 722
#     det.add_rotation(90, 'z')

In [ ]:
det.plot_hits_1d()

In [ ]:
print(det.x_hits.shape)
print(det.y_hits.shape)

In [ ]:
det.plot_centroids_2d()

In [ ]:
det.plot_xy_hit_map()

In [ ]:
det.get_det_clusters()

In [ ]:
det.dream_data.correct_for_fine_timestamps()

sigma_x, sigma_x_err = det.dream_data.plot_event_time_maxes(max_channel=True, channels=np.arange(0, int(256 / 2)), min_amp=None, plot=True)
plt.title(f'Time of Max for X (Top) Strips')

sigma_y, sigma_y_err = det.dream_data.plot_event_time_maxes(max_channel=True, channels=np.arange(int(256 / 2), 256), min_amp=None, plot=True)
plt.title(f'Time of Max for Y (Bottom) Strips')

In [ ]:
min_amp = 600
sigma_x, sigma_x_err = det.dream_data.plot_event_time_maxes(max_channel=True, channels=np.arange(0, int(256 / 2)), min_amp=min_amp, plot=True)
plt.title(f'Time of Max for X (Top) Strips Min Amp {min_amp}')

sigma_y, sigma_y_err = det.dream_data.plot_event_time_maxes(max_channel=True, channels=np.arange(int(256 / 2), 256), min_amp=min_amp, plot=True)
plt.title(f'Time of Max for Y (Bottom) Strips Min Amp {min_amp}')